In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tweepy
import psycopg2
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
# Keys
consumer_key = 'Eb7Q832xETPncAvbICrimtbgQ'
consumer_secret = 'myJ84Ry5Zete9zgqJtQb7pZK8fw9oAqF7CssGQAj0PXVdH5pP0'
access_token = '2760228716-kr6JayzHlTNbEf0JGmhaGdJzsVhXWqRU83nolpK'
access_secret = 'h7rgFGE0IrfQQXxotXuvhfAhZJq08aD7em6V5fo1fEO22'

In [3]:
# Twitter api
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
# Connect to database
conn_new = psycopg2.connect(dbname='conn_new2', user='postgres')
cursor = conn_new.cursor()

In [5]:
# Create table
cursor.execute("CREATE TABLE IF NOT EXISTS tweets (id SERIAL PRIMARY KEY, user_name VARCHAR, media BOOL, retweetcount INT, favcount INT, created_at TIMESTAMP, tweet text NOT NULL, location VARCHAR);")
conn_new.commit()

In [ ]:
# Insert into table
try:
    for tweet in tweepy.Cursor(api.search,q="#nba -filter:retweets",tweet_mode='extended',count=100).items(17790):
        if 'media' in tweet.entities:
            media_bool = True
        else:
            media_bool = False
        cursor.execute("INSERT INTO tweets (user_name, media, retweetcount, favcount, created_at, tweet, location) VALUES (%s, %s, %s, %s, %s, %s, %s);", (tweet.author.screen_name, media_bool, tweet.retweet_count, tweet.favorite_count, tweet.created_at, tweet.full_text, tweet.user.location))
        conn_new.commit()        
except Exception as e:
         print(e)
        
cursor.close()

In [ ]:
# Read from database
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@localhost:5432/conn_new2')
df = pd.read_sql_query('select * FROM tweets',conn_new)

In [ ]:
# Close connection to database
conn_new.close()

In [ ]:
df

In [ ]:
total_tweets = len(df)

In [ ]:
total_tweets

In [ ]:
unique_users = df.user_name.nunique()

In [ ]:
unique_users

In [ ]:
media_tweets_num = len(df[df.media == True])

In [ ]:
media_tweets_num

In [ ]:
most_retweeted = df[df.retweetcount == max(df.retweetcount)]

In [ ]:
most_retweeted

In [ ]:
most_fav = df[df.favcount == max(df.favcount)]

In [ ]:
most_fav

In [ ]:
# Replace blank locations
df['location'] = df.location.replace('', 'No location provided')

In [ ]:
df.groupby('location').size().sort_values(ascending=False)

In [ ]:
df.groupby('user_name').size().sort_values(ascending=False).head()

In [ ]:
df.groupby('user_name')['retweetcount'].sum().sort_values(ascending = False).head(5)

In [ ]:
df.groupby('user_name')['favcount'].sum().sort_values(ascending = False).head(5)

In [ ]:
plotly.tools.set_credentials_file(username='gunajit23', api_key='ULuUce6nAp56xGWMIH2W')

In [ ]:
till_hour = pd.to_datetime(df.created_at.dt.strftime('%Y-%m-%d-%H'))

In [ ]:
trace = go.Scatter(
    x = df.groupby(till_hour).size().index,
    y = df.groupby(till_hour).size().values
)

data = [trace]

py.iplot(data, filename='tweets_num')

In [ ]:
trace = go.Scatter(
    x = df[df.media == True].groupby(till_hour).size().index,
    y = df[df.media == True].groupby(till_hour).size().values
)

data = [trace]

py.iplot(data, filename='media_tweets_num')